In [ ]:
'''
Author - Gaurav Kothyari
Date   - 29-09-2020
Version - Contexual_error_correction - version1
'''

!pip install wikipedia
!pip install spacy
import spacy
sp = spacy.load('en_core_web_sm')
# import spacy
import wikipedia

#text data for understanding the context of the data
artificial_intelligence = wikipedia.page("Artificial Intelligence").content
machine_learning = wikipedia.page("machine learning").content
Deep_learning = wikipedia.page("Deep learning").content
Data_science = wikipedia.page("Data science").content
computer_Programming = wikipedia.page("computer Programming").content
Multiprocessing = wikipedia.page("Multiprocessing").content
software_company = wikipedia.page("software company").content
software_indistry = wikipedia.page("software indistry").content
recruitment = wikipedia.page("recruitment").content
problem_solving = wikipedia.page("problem solving").content
Software_deployment = wikipedia.page("Software deployment").content
Software_development = wikipedia.page("Software development").content
Employee_relationship_management = wikipedia.page("Employee relationship management").content
Business_relations = wikipedia.page("Business relations").content
Customer_service = wikipedia.page("Customer service").content
Workplace_harassment = wikipedia.page("Workplace harassment").content
Information_Technology_in_India = wikipedia.page("Information Technology in India").content

from nltk.tokenize import sent_tokenize

dataset_gk = []


artificial_intelligence =     [s.text for s in sp(artificial_intelligence).sents]
machine_learning =            [s.text for s in sp(machine_learning).sents]
Deep_learning =               [s.text for s in sp(Deep_learning).sents]
Data_science =                [s.text for s in sp(Data_science).sents]
computer_Programming =         [s.text for s in sp(computer_Programming).sents]
Multiprocessing =          [s.text for s in sp(Multiprocessing).sents]
software_company =         [s.text for s in sp(software_company).sents]
software_indistry =       [s.text for s in sp(software_indistry).sents]
recruitment =           [s.text for s in sp(recruitment).sents]
problem_solving =        [s.text for s in sp(problem_solving).sents]
Software_deployment =      [s.text for s in sp(Software_deployment).sents]
Software_development =      [s.text for s in sp(Software_development).sents]
Employee_relationship_management =    [s.text for s in sp(Employee_relationship_management).sents]
Business_relations =             [s.text for s in sp(Business_relations).sents]
Customer_service =                [s.text for s in sp(Customer_service).sents]
Workplace_harassment =          [s.text for s in sp(Workplace_harassment).sents]
Information_Technology_in_India =  [s.text for s in sp(Information_Technology_in_India).sents]



dataset_gk.extend(artificial_intelligence)
dataset_gk.extend(machine_learning)
dataset_gk.extend(Deep_learning)
dataset_gk.extend(Data_science)
dataset_gk.extend(computer_Programming)
dataset_gk.extend(Multiprocessing)
dataset_gk.extend(software_company)
dataset_gk.extend(software_indistry)
dataset_gk.extend(recruitment)
dataset_gk.extend(problem_solving)
dataset_gk.extend(Software_deployment)
dataset_gk.extend(Software_development)
dataset_gk.extend(Employee_relationship_management)
dataset_gk.extend(Business_relations)
dataset_gk.extend(Customer_service)
dataset_gk.extend(Workplace_harassment)
dataset_gk.extend(Information_Technology_in_India)


# print(dataset_gk)

In [ ]:
# lets do the data preprocessing.
# !pip install -U spacy
import spacy
sp = spacy.load('en_core_web_sm')


import re

def preprocess1(str1):
    
    # apply some regular expression.
    tr = str1
    #remove non word character
    pattern1 = r"\W"
    str1 = re.sub(pattern1,' ',str1)
    
    #remove isolated character
    pattern2 =r'\s+[a-zA-Z]\s+'
    str1 = re.sub(pattern2,' ',str1)
    
    #remove first single character
    pattern3 =r"^[a-zA-Z]\s+"
    str1 = re.sub(pattern3,' ',str1)
    
    #remove multiple spaces with single space
    pattern4 = r"\s+"
    str1 = re.sub(pattern4,' ',str1,flags=re.I)
    
    #lowercase string.lower
    str1 = str1.lower()
    
    #split the sentence into tokens using space
    #tokens= str1.split(' ')
    tokens = [w for w in sp(str1)]
    
    all_stopwords = sp.Defaults.stop_words
    s_words = [wd1.lemma_ for wd1 in tokens if wd1.text in all_stopwords]
    
    n_s_words = [wd for wd in tokens if wd.text not in all_stopwords]
    
    
    #lemmatization
    tokens = [word.lemma_ for word in n_s_words]
    
    #join the list
    tokens_sent = ' '.join(tokens)
    
    token_word = tokens + s_words
    
    #print(token_word)
    
    return (token_word,tokens_sent)
    

In [ ]:
#get the data
my_dictionary = []
data1 = []
for str1 in dataset_gk:
    if str1!= '':
        #print(str1)
        temp_out = preprocess1(str1)
        #print(temp_out)
        data1.append(temp_out[1])
        my_dictionary.extend(temp_out[0])
        
#Again tokenize the string
sentence_token = [[words.text for words in sp(cl_sent) if words.text not in [' ', '  ']] for cl_sent in data1]
    
# print(sentence_token[0:10])
# print(my_dictionary[0:10])


In [ ]:
#lets write the hyperparameter for model 
embedding_size = 60
window_size = 40
min_word = 5
down_sampling = 1e-2

from gensim.models.fasttext import FastText

ft_model = FastText(sentence_token,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=1,
                      iter=100)

ft_model.wv.save_word2vec_format('model.bin')

In [ ]:
#create a word count dictionary
word_count = {}

for x in my_dictionary:
    if x not in ['',' ',]:
        str1 = str(x).lower()
        if str1 in word_count.keys():
            word_count[str1] = word_count[str1] + 1
        else:
            word_count[str1] = 1


for sn,d in enumerate(word_count.items()):
    k,v = d
    if sn <10:
        print(k,v)

words = word_count
len(words)

In [ ]:
# original from https://github.com/cbaziotis/ekphrasis/blob/master/ekphrasis/classes/spellcorrect.py
# improved it

import re
from collections import Counter

class SpellCorrector:
    """
    The SpellCorrector extends the functionality of the Peter Norvig's
    spell-corrector in http://norvig.com/spell-correct.html
    """

    def __init__(self):
        """
        :param corpus: the statistics from which corpus to use for the spell correction.
        """
        super().__init__()
        self.WORDS = words
        self.N = sum(self.WORDS.values())
        
    @staticmethod
    def tokens(text):
        return REGEX_TOKEN.findall(text.lower())

    def P(self, word):
        """
        Probability of `word`.
        """
        return self.WORDS[word] / self.N

    def most_probable(self, words):
        _known = self.known(words)
        if _known:
            return max(_known, key=self.P)
        else:
            return []

    @staticmethod
    def edit_step(word):
        """
        All edits that are one edit away from `word`.
        """
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        """
        All edits that are two edits away from `word`.
        """
        return (e2 for e1 in self.edit_step(word)
                for e2 in self.edit_step(e1))

    def known(self, words):
        """
        The subset of `words` that appear in the dictionary of WORDS.
        """
        return set(w for w in words if w in self.WORDS)

    def edit_candidates(self, word, assume_wrong=False, fast=True):
        """
        Generate possible spelling corrections for word.
        """

        if fast:
            ttt = self.known(self.edit_step(word)) or {word}
        else:
            ttt = self.known(self.edit_step(word)) or self.known(self.edits2(word)) or {word}
        
        ttt = self.known([word]) | ttt
        return list(ttt)

In [ ]:
corrector = SpellCorrector()
possible_states = corrector.edit_candidates('wh')
possible_states
ft_model.wv.most_similar(['wh'], topn=3)

In [ ]:
import spacy
import re
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

#This class checks the context word if given and use the word vector for getting the contexually correct suggestion.
    
def score_normalise(sorted_dict):
    
    n_s_d = {}
    for i in sorted_dict:
        k,v = i
        dict_length = len(words)
        frequency = words[k]/dict_length
        factor = frequency*v
        n_s_d[k] = factor
    #print('n_s_d',n_s_d)
    return n_s_d
        

def context_word(un_word):
    full_out_str = []
    
    
    for word2 in un_word:
       
        corrector = SpellCorrector()
        
        #get the candidate for this error
        possible_states = corrector.edit_candidates(word2)
        if possible_states[0] == word2:
            possible_states = []
        
        #get the possible states
        m_out = [(item[0],item[1]) for item in ft_model.wv.most_similar([word2], topn=3)]
        
        for f in m_out:   
            if f[1] >= 0.74:
                possible_states.append(f[0])
        
        score_dict = {}
        #print('ps',possible_states)
        
        if len(possible_states) > 0:
            
            
            for c in list(set(possible_states)):
                score = ft_model.wv.similarity(w1=word2, w2=c)
                score_dict[c] = score

            sorted_dict = [(k,v) for k,v in sorted(score_dict.items(),key = lambda x:x[1])]
            
            n_score_dict = score_normalise(sorted_dict)
            
            n_score_dict = [(k,v) for k,v in sorted(n_score_dict.items(),key = lambda x:x[1])]
            
            if len(n_score_dict)>0:
                out_str = n_score_dict[-1][0]
            else:
                out_str = word2
        else:
            out_str = word2

        #print('out for word is',out_str)
        full_out_str.append(out_str)
        
    return full_out_str
    
    
    

def no_context(word1):
    
    word1 = word1[0]
    out_str = []
    
    #get the candidate for this error
    corrector = SpellCorrector()
    possible_states = corrector.edit_candidates(word1)
    
    
    if possible_states[0] == word1:
        
        possible_states = []
    
    model_o = [(item[0],item[1]) for item in ft_model.wv.most_similar([word1], topn=3)]
    
    for f in model_o:
        if f[1] > 0.74:
            possible_states.append(f[0])
            
    #print(possible_states)
    score_dict = {}
    
    if len(set(possible_states)) == 1:
        out_str = possible_states   
        
    else:  
        for c in list(set(possible_states)):
            
            score = ft_model.wv.similarity(w1=word1, w2=c)
            score_dict[c] = score
            
        sorted_dict = [(k,v) for k,v in sorted(score_dict.items(),key = lambda x:x[1])]
        
        n_score_dict = score_normalise(sorted_dict)
        
        n_score_dict = [(k,v) for k,v in sorted(n_score_dict.items(),key = lambda x:x[1])]
        
        
        if len(n_score_dict)>0:
            out_str = [n_score_dict[-1][0]]
        else:
            out_str = [word1]
        
    return out_str
        
        
    
def test_model(test_str):
    
    sp = spacy.load('en_core_web_sm')
    all_stopwords = sp.Defaults.stop_words
    
    test_word = [w for w in sp(test_str)]
    
    un_words = []
    
    un_lemma_word = []
    
    known_word = []
    
    for w1 in test_word:
        
        if w1.text.lower() not in all_stopwords:
            
            #get the lemma 
            w_lemma = w1.lemma_
            if w_lemma.lower() not in word_count.keys():
                un_words.append(w1.text)
                un_lemma_word.append(w_lemma.lower()) 
            else:
                known_word.append(w_lemma.lower())
                
            
    
    text_msk = ''
    unknown_word = []
    output_str = ''
    output_list = []
    
    
    
    if len(un_lemma_word) > 0: 
        
        #check if length:
        if len(test_str) <= 2:
            output_list = no_context(un_words)


        if len(test_str) > 2:
            output_list = context_word(un_words)
        
        #print('output is ',output_list)
        
        #substitute the value
        for i in range(len(un_words)):
            
            #print(un_words[i],output_list[i])
            test_str = re.sub(un_words[i],output_list[i],test_str)
            
    print('corrected string is -> ', test_str)
            


In [ ]:
#Here you can test the model using your own string

test_str = 'whre'

print('test_string is ->',test_str)

test_model(test_str.strip(' '))
